<!-- 
Generated by ALAIN (Applied Learning AI Notebooks) on 2025-09-14
Teacher Model: GPT-OSS-20B
Provider: openai-compatible
Target Model: gpt-oss:20b
Learn more: https://github.com/daniel-p-green/alain-ai-learning-platform/
-->


# Getting Started with GPT-OSS-20B

Learn what GPT-OSS-20B is, its common uses, and how to run a simple chat completion using an OpenAI‑compatible API.

> Provider: `openai-compatible`  •  Model: `gpt-oss:20b`
Runtime: openai-compatible

This notebook was generated by ALAIN. It calls AI models via OpenAI-compatible APIs (no arbitrary code).

---

### Reproducibility Tips
- Avoid network access in core cells.
- Seed randomness where applicable (e.g., numpy, random).
- Pin package versions in your own environment if needed.
- Set `OPENAI_BASE_URL` and `OPENAI_API_KEY` via env (or Colab userdata).
- Widgets optional: text-based MCQs are provided if widgets are unavailable.


In [ ]:
# Create requirements.txt for reproducible environment
%%writefile requirements.txt
openai>=1.34.0
ipywidgets>=8.0.0
requests>=2.31.0
python-dotenv>=1.0.0
numpy>=1.24.0
pandas>=2.0.0


In [ ]:
# Create .env.example template
%%writefile .env.example
# Copy this file to .env and fill in your actual values
OPENAI_API_KEY=your_api_key_here
OPENAI_BASE_URL=http://localhost:1234/v1
POE_API_KEY=your_poe_key_here
# For local models (LM Studio/Ollama), any non-empty string works for API_KEY


In [ ]:
# Install dependencies from requirements.txt
%pip install -q -r requirements.txt
print('✅ Dependencies installed successfully')


In [ ]:
# Configure OpenAI-compatible client
import os
from dotenv import load_dotenv
from getpass import getpass
# Load environment variables from .env file if it exists
load_dotenv()
# Try to read secrets from Colab userdata if available
try:
  from google.colab import userdata  # type: ignore
  _poe = userdata.get('POE_API_KEY')
  _openai = userdata.get('OPENAI_API_KEY')
except Exception:
  _poe = None; _openai = None
PROVIDER = "openai-compatible"  # "poe" or "openai-compatible"
os.environ.setdefault("OPENAI_BASE_URL", "http://localhost:1234/v1")
# Set your API key. For Poe, set POE_API_KEY; for local (LM Studio/Ollama) any non-empty string works.
os.environ.setdefault("OPENAI_API_KEY", _poe or _openai or os.getenv("POE_API_KEY") or os.getenv("OPENAI_API_KEY") or "")
# Local-friendly defaults to avoid prompting beginners
if (PROVIDER == 'openai-compatible') and not os.environ.get('OPENAI_API_KEY'):
  base = os.environ.get('OPENAI_BASE_URL','')
  if 'localhost:1234' in base or '127.0.0.1:1234' in base:
    os.environ['OPENAI_API_KEY'] = 'lm-studio'
  elif 'localhost:11434' in base or '127.0.0.1:11434' in base:
    os.environ['OPENAI_API_KEY'] = 'ollama'
# Fallback interactive prompt if still missing
if not os.environ.get('OPENAI_API_KEY'):
  os.environ['OPENAI_API_KEY'] = getpass('Enter API key (input hidden): ')
# OPENAI_BASE_URL and OPENAI_API_KEY environment variables are set above


In [ ]:
# Pre-flight check: verify API connectivity
from openai import OpenAI
import os, sys
base = os.environ.get('OPENAI_BASE_URL')
key = os.environ.get('OPENAI_API_KEY')
if not base or not key:
    print('❌ Missing OPENAI_BASE_URL or OPENAI_API_KEY. Set them above.')
else:
    try:
        client = OpenAI(base_url=base, api_key=key)
        # lightweight call: list models or small completion
        ok = False
        try:
            _ = client.models.list()
            ok = True
        except Exception:
            # Fallback to a 1-token chat call
            _ = client.chat.completions.create(model="${meta.model}", messages=[{"role":"user","content":"ping"}], max_tokens=1)
            ok = True
        if ok:
            print('✅ API key is working and connected to provider.')
    except Exception as e:
        print('❌ Connection failed. Please check your API key and base URL.\n', e)


In [ ]:
# Quick smoke test
from openai import OpenAI
import os
base = os.environ.get('OPENAI_BASE_URL')
key = os.environ.get('OPENAI_API_KEY')
assert base and key, 'Please set OPENAI_BASE_URL and OPENAI_API_KEY env vars'
client = OpenAI(base_url=base, api_key=key)
resp = client.chat.completions.create(model="gpt-oss:20b", messages=[{"role":"user","content":"Hello from ALAIN"}], max_tokens=32)
print(resp.choices[0].message.content)


In [ ]:
# Tool Use demo (LM Studio recommended)
# This shows how to pass OpenAI-compatible 'tools' and handle tool_calls.
from openai import OpenAI
import os, json, datetime
client = OpenAI(base_url=os.environ['OPENAI_BASE_URL'], api_key=os.environ['OPENAI_API_KEY'])
tools = [{
  'type': 'function',
  'function': {
    'name': 'get_current_time',
    'description': 'Return the current local time as an ISO string',
    'parameters': { 'type': 'object', 'properties': {} }
  }
}]
messages = [{"role":"user","content":"Tell me a joke, then use a tool to tell the current time."}]
resp = client.chat.completions.create(model="gpt-oss:20b", messages=messages, tools=tools)
m = resp.choices[0].message
print('finish_reason:', resp.choices[0].finish_reason)
if getattr(m, 'tool_calls', None):
    for tc in m.tool_calls:
        if tc.function and tc.function.name == 'get_current_time':
            tool_out = datetime.datetime.now().isoformat()
            # Feed tool result back to the model for a final answer
            messages.append({'role': 'assistant', 'tool_calls': [tc]})
            messages.append({'role': 'tool', 'content': tool_out})
            final = client.chat.completions.create(model="gpt-oss:20b", messages=messages)
            print(final.choices[0].message.content)
else:
    # No tool call requested; show normal content
    print(m.content)


## Step 1: What is GPT‑OSS‑20B?




In [ ]:
# Run the step prompt using the configured provider
PROMPT = """

"""
from openai import OpenAI
import os
client = OpenAI(base_url=os.environ['OPENAI_BASE_URL'], api_key=os.environ['OPENAI_API_KEY'])
resp = client.chat.completions.create(model="gpt-oss:20b", messages=[{"role":"user","content":PROMPT}], temperature=0.7, max_tokens=400)
print(resp.choices[0].message.content)


## Step 1: Typical Use Cases




In [ ]:
# Run the step prompt using the configured provider
PROMPT = """

"""
from openai import OpenAI
import os
client = OpenAI(base_url=os.environ['OPENAI_BASE_URL'], api_key=os.environ['OPENAI_API_KEY'])
resp = client.chat.completions.create(model="gpt-oss:20b", messages=[{"role":"user","content":PROMPT}], temperature=0.7, max_tokens=400)
print(resp.choices[0].message.content)


## Step 1: Set Up API Access




In [ ]:
# Run the step prompt using the configured provider
PROMPT = """

"""
from openai import OpenAI
import os
client = OpenAI(base_url=os.environ['OPENAI_BASE_URL'], api_key=os.environ['OPENAI_API_KEY'])
resp = client.chat.completions.create(model="gpt-oss:20b", messages=[{"role":"user","content":PROMPT}], temperature=0.7, max_tokens=400)
print(resp.choices[0].message.content)


## Step 1: Write a Simple Chat Script




In [ ]:
# Run the step prompt using the configured provider
PROMPT = """

"""
from openai import OpenAI
import os
client = OpenAI(base_url=os.environ['OPENAI_BASE_URL'], api_key=os.environ['OPENAI_API_KEY'])
resp = client.chat.completions.create(model="gpt-oss:20b", messages=[{"role":"user","content":PROMPT}], temperature=0.7, max_tokens=400)
print(resp.choices[0].message.content)


## Step 1: Run and Test




In [ ]:
# Run the step prompt using the configured provider
PROMPT = """

"""
from openai import OpenAI
import os
client = OpenAI(base_url=os.environ['OPENAI_BASE_URL'], api_key=os.environ['OPENAI_API_KEY'])
resp = client.chat.completions.create(model="gpt-oss:20b", messages=[{"role":"user","content":PROMPT}], temperature=0.7, max_tokens=400)
print(resp.choices[0].message.content)
